## **LOCACIONES PLANTAS CLIENTES**

In [10]:
from docplex.mp.model import Model
import pandas

In [11]:
df_costo_cliente_planta = pandas.read_excel("./data.xlsx", sheet_name="COSTO CLIENTE-PLANTA")
df_clientes = pandas.read_excel("./data.xlsx", sheet_name="DEMANDAS CLIENTES")
df_plantas = pandas.read_excel("./data.xlsx", sheet_name="COSTO Y CAPACIDAD PLANTAS")

**Conjuntos**

$$ P: \text{Plantas} = \{1, 2,...,30\} $$
$$ C: \text{Clientes} = \{1, 2,...,50\} $$

In [12]:
PLANTAS = df_plantas["Planta"].to_list()
CLIENTES = df_clientes["Cliente"].to_list()

**Parámetros**

$$ \text{CostoApertura}_{p}: \text{Costo de abrir la planta } p\in P $$
$$ \text{Capacidad}_{p}: \text{Capacidad máxima de la planta } p\in P $$
$$ \text{Demanda}_{c}: \text{Demanda del cliente } c\in C $$
$$ \text{CostoAsignacion}_{p,c}: \text{Costo de asignar la planta } p\in P \text{ al cliente } c\in C $$

In [13]:
COSTO_CLIENTE_PLANTA = df_costo_cliente_planta.set_index(["Planta", "Cliente"])['Costo de asignar un cliente a un planta'].to_dict()
DEMANDA_CLIENTE = df_clientes.set_index(["Cliente"])["Demanda del cliente"].to_dict()
COSTO_ABRIR_PLANTA = df_plantas.set_index(["Planta"])["Costo de abrir planta"].to_dict()
CAPACIDAD_PLANTA = df_plantas.set_index(["Planta"])["Capacidad máxima"].to_dict()

In [14]:
model = Model()

**Variables**

$$ X_{p} = \left\{ \begin{array}{cl}
1: \text{Si abro la planta } p \in P \\
0: \text{dlc}
\end{array} \right. $$
$$ Y_{pc} = \left\{ \begin{array}{cl}
1: \text{Si asigno el cliente } c \in C \text{ a la planta } p \in P \\
0: \text{dlc}
\end{array} \right. $$

In [15]:
x = model.binary_var_dict(PLANTAS, name="x")
y = model.binary_var_dict([(planta, cliente) for planta in PLANTAS for cliente in CLIENTES], name="y")

**Función Objetivo**

$$ \text{minimizar } \text{FO} = \sum_{p\in P} \text{CAP}_{p} \times X_{p} + \sum_{p\in P} \sum_{c\in C} \text{CCP}_{pc} \times Y_{pc} $$

In [16]:
costo_abrir_plantas = model.sum(x[planta] * COSTO_ABRIR_PLANTA[planta] for planta in PLANTAS)
costo_cliente_planta = model.sum(y[(planta, cliente)] * COSTO_CLIENTE_PLANTA[(planta, cliente)] for planta in PLANTAS for cliente in CLIENTES)
model.minimize(costo_abrir_plantas + costo_cliente_planta)

model.add_kpi(costo_abrir_plantas, "costo_abrir_plantas")
model.add_kpi(costo_cliente_planta, "costo_cliente_planta")

DecisionKPI(name=costo_cliente_planta,expr=29y_1_1+45y_1_2+47y_1_3+81y_1_4+y_1_5+38y_1_6+32y_1_7+43y_1_8+64..)

**Restricciones**

$$ \forall p\in P \, \forall c\in C: Y_{pc} \le X_{p} $$

$$ \forall p\in P: \sum_{c\in C} Y_{pc} \times D_{c} \le \text{CAPAC}_{p} $$

$$ \forall c\in C: \sum_{p\in P} Y_{pc} = 1 $$

$$ \forall p\in P: \sum_{c\in C} Y_{pc} \ge 3 X_{p}$$

$$ \forall p\in P: \sum_{c\in C} Y_{pc} \times D_{c} \ge 0.9 * \text{CAPAC}_{p} X_{p} $$

In [17]:
for planta in PLANTAS:
    for cliente in CLIENTES:
        model.add_constraint(y[(planta, cliente)] <= x[planta], ctname=f"ct1_{planta}_{cliente}")

for planta in PLANTAS:
    model.add_constraint(model.sum(y[(planta, cliente)] * DEMANDA_CLIENTE[cliente] for cliente in CLIENTES) <= CAPACIDAD_PLANTA[planta], ctname=f"ct2_{planta}")

for cliente in CLIENTES:
    model.add_constraint(model.sum(y[(planta, cliente)] for planta in PLANTAS) == 1, ctname=f"ct3_{cliente}")

for planta in PLANTAS:
    model.add_constraint(model.sum(y[(planta, cliente)] for cliente in CLIENTES) >= 3 * x[planta], ctname=f"ct4_{planta}")

for planta in PLANTAS:
    model.add_constraint(model.sum(y[(planta, cliente)] * DEMANDA_CLIENTE[cliente] for cliente in CLIENTES) >= 0.9 * CAPACIDAD_PLANTA[planta] * x[planta], ctname=f"ct5_{planta}")


In [18]:
model.export(f"./locaciones.lp")

'./locaciones.lp'

In [ ]:
model.set_time_limit(20)

In [10]:
solution = model.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 91 rows and 74 columns.
MIP Presolve added 17 rows and 17 columns.
MIP Presolve modified 981 coefficients.
Reduced MIP has 1036 rows, 963 columns, and 4807 nonzeros.
Reduced MIP has 946 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (8.21 ticks)
Probing time = 0.00 sec. (2.16 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1036 rows, 963 columns, and 4807 nonzeros.
Reduced MIP has 946 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (4.44 ticks)
Probing time = 0.00 sec. (2.16 ticks)
Clique table members: 924.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.08 sec. (35.75 ticks)

        Nodes                                         Cut

In [ ]:
solution

docplex.mp.solution.SolveSolution(obj=40101,values={x_3:1,x_4:1,x_6:1,x_..

In [ ]:
model.get_solve_status()

<JobSolveStatus.FEASIBLE_SOLUTION: 1>

In [ ]:
model.objective_value

40101.0

In [ ]:
y[(3, 1)].solution_value

0

In [ ]:
PLANTAS_ABIERTAS = []
for planta in PLANTAS:
    if x[planta].solution_value > 0.99:
        print(f"=== Planta {planta} ====")
        print(f"Capacidad Planta: {CAPACIDAD_PLANTA[planta]}")
        print(f"Cantidad Enviada: {sum(round(y[(planta, cliente)].solution_value, 0) * DEMANDA_CLIENTE[cliente] for cliente in CLIENTES)}")
        CLIENTES_ATENDIDOS = []
        for cliente in CLIENTES:
            if y[(planta, cliente)].solution_value > 0.99:
                print(cliente, end="  ")
                CLIENTES_ATENDIDOS.append(cliente)
        print("\n")
        PLANTAS_ABIERTAS.append(
            {
                "ID": planta,
                "CAPACIDAD_PLANTA": CAPACIDAD_PLANTA[planta],
                "CANTIDAD_ENVIADA": sum(round(y[(planta, cliente)].solution_value, 0) * DEMANDA_CLIENTE[cliente] for cliente in CLIENTES),
                "CLIENTES_ATENDIDOS": CLIENTES_ATENDIDOS,
            }
        )

=== Planta 3 ====
Capacidad Planta: 32
Cantidad Enviada: 32.0
10  37  38  

=== Planta 4 ====
Capacidad Planta: 52
Cantidad Enviada: 52.0
8  18  33  

=== Planta 6 ====
Capacidad Planta: 95
Cantidad Enviada: 94.0
3  4  30  44  

=== Planta 7 ====
Capacidad Planta: 29
Cantidad Enviada: 29.0
9  35  41  

=== Planta 9 ====
Capacidad Planta: 80
Cantidad Enviada: 80.0
19  46  48  

=== Planta 10 ====
Capacidad Planta: 97
Cantidad Enviada: 97.0
13  25  47  

=== Planta 11 ====
Capacidad Planta: 23
Cantidad Enviada: 23.0
11  22  23  

=== Planta 12 ====
Capacidad Planta: 85
Cantidad Enviada: 85.0
5  6  26  43  

=== Planta 13 ====
Capacidad Planta: 72
Cantidad Enviada: 72.0
21  28  31  

=== Planta 14 ====
Capacidad Planta: 62
Cantidad Enviada: 62.0
12  27  40  

=== Planta 15 ====
Capacidad Planta: 48
Cantidad Enviada: 47.0
15  39  49  

=== Planta 16 ====
Capacidad Planta: 100
Cantidad Enviada: 100.0
7  14  20  32  45  

=== Planta 17 ====
Capacidad Planta: 77
Cantidad Enviada: 77.0
17  29 

In [ ]:
PLANTAS_ABIERTAS

[{'ID': 3,
  'CAPACIDAD_PLANTA': 32,
  'CANTIDAD_ENVIADA': 32.0,
  'CLIENTES_ATENDIDOS': [10, 37, 38]},
 {'ID': 4,
  'CAPACIDAD_PLANTA': 52,
  'CANTIDAD_ENVIADA': 52.0,
  'CLIENTES_ATENDIDOS': [8, 18, 33]},
 {'ID': 6,
  'CAPACIDAD_PLANTA': 95,
  'CANTIDAD_ENVIADA': 94.0,
  'CLIENTES_ATENDIDOS': [3, 4, 30, 44]},
 {'ID': 7,
  'CAPACIDAD_PLANTA': 29,
  'CANTIDAD_ENVIADA': 29.0,
  'CLIENTES_ATENDIDOS': [9, 35, 41]},
 {'ID': 9,
  'CAPACIDAD_PLANTA': 80,
  'CANTIDAD_ENVIADA': 80.0,
  'CLIENTES_ATENDIDOS': [19, 46, 48]},
 {'ID': 10,
  'CAPACIDAD_PLANTA': 97,
  'CANTIDAD_ENVIADA': 97.0,
  'CLIENTES_ATENDIDOS': [13, 25, 47]},
 {'ID': 11,
  'CAPACIDAD_PLANTA': 23,
  'CANTIDAD_ENVIADA': 23.0,
  'CLIENTES_ATENDIDOS': [11, 22, 23]},
 {'ID': 12,
  'CAPACIDAD_PLANTA': 85,
  'CANTIDAD_ENVIADA': 85.0,
  'CLIENTES_ATENDIDOS': [5, 6, 26, 43]},
 {'ID': 13,
  'CAPACIDAD_PLANTA': 72,
  'CANTIDAD_ENVIADA': 72.0,
  'CLIENTES_ATENDIDOS': [21, 28, 31]},
 {'ID': 14,
  'CAPACIDAD_PLANTA': 62,
  'CANTIDAD_ENVIADA'